# Pre-processing News Data

## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau 

- data/news_vnexpress/

    - Kinh tế: 
        - bài báo 1.txt 
        - bài báo 2.txt 
    - Pháp luật
        - bài báo 3.txt 
        - bài báo 4.txt 

In [3]:
INPUT = 'D:\HUST\Tài liệu thầy Ngô Linh\ML-homeworks\Hw1\\news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

In [5]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
    n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
du-lich: 54
giai-tri: 201
giao-duc: 105
khoa-hoc: 144
kinh-doanh: 262
phap-luat: 59
suc-khoe: 162
the-thao: 173
thoi-su: 59
-------------------------
Tổng số văn bản: 1339


In [6]:
data_train = load_files(container_path=INPUT, encoding="utf-8")

In [11]:
# load data
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')


print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['D:\\HUST\\Tài liệu thầy Ngô Linh\\ML-homeworks\\Hw1\\news_vnexpress\\khoa-hoc\\00133.txt']
[4]
['Mời độc giả đặt câu hỏi tại đây\n']

Tổng số  văn bản: 1339


## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

* Bạn cần viết đoạn mã tương ứng trong cell bên dưới. Theo các bước được gợi ý

In [32]:
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = file.readlines()
stop_words = [line.strip().replace(' ', '_') for line in stop_words]

In [33]:
stop_words[:10]

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy']

In [29]:
data_train.data[1]

'(Nguồn và ảnh: Trung tâm cấy tóc New Hair)\n'

In [34]:
counter = CountVectorizer(stop_words=stop_words)
model_rf_preprocess = Pipeline([('counter', counter),
                    ('tfidf', TfidfTransformer()),
                    ])

In [35]:
data_preprocessed = model_rf_preprocess.fit_transform(data_train.data)

In [36]:
X = data_preprocessed # thuoc tinh
Y = data_train.target #nhan

print(f"\nSố lượng từ trong từ điển: {len(counter.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")


Số lượng từ trong từ điển: 12796
Kích thước dữ liệu sau khi xử lý: (1339, 12796)
Kích thước nhãn tương ứng: (1339,)


In [37]:
print(X[100].toarray())
print(Y[100])

[[0.         0.         0.         ... 0.         0.14048828 0.        ]]
5


In [38]:
sum(sum(X[100].toarray() != 0))

289

In [39]:
print(X[100])

  (0, 12794)	0.14048828324700804
  (0, 12724)	0.051226678060487627
  (0, 12714)	0.034379239518190156
  (0, 12705)	0.024927343279465615
  (0, 12697)	0.03935911209707954
  (0, 12692)	0.013885134230282647
  (0, 12691)	0.02076954755505395
  (0, 12672)	0.03173992101554847
  (0, 12646)	0.04268947993761032
  (0, 12643)	0.030193779677554416
  (0, 12629)	0.024173036345759045
  (0, 12626)	0.01928809379275951
  (0, 12624)	0.3318224864003995
  (0, 12617)	0.08000423234784886
  (0, 12591)	0.07519534686809994
  (0, 12584)	0.03876774373554222
  (0, 12566)	0.033240367004725005
  (0, 12558)	0.03206234356763185
  (0, 12547)	0.04575286598942787
  (0, 12535)	0.05488370325838488
  (0, 12521)	0.09355442947181113
  (0, 12517)	0.03883219864696093
  (0, 12509)	0.017786174579851665
  (0, 12454)	0.07589970050190288
  (0, 12272)	0.02125953768208212
  :	:
  (0, 2170)	0.029508397725910254
  (0, 2159)	0.016084504788746505
  (0, 2140)	0.015661963686282587
  (0, 2135)	0.04322051581452054
  (0, 2111)	0.02577563465433511